## Data Input

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras.models import Sequential,Model
from keras.layers import Input,Dense,Activation
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE

Using TensorFlow backend.


In [3]:
LMS = pd.read_excel('/home/soham/hackerearth_bankloans/93fe009c-2-Foreclosure-Prediction-Dataset_/LMS_31JAN2019.xlsx');

In [4]:
test = pd.read_csv('/home/soham/hackerearth_bankloans/93fe009c-2-Foreclosure-Prediction-Dataset_/test_foreclosure.csv')

In [5]:
train = pd.read_csv('/home/soham/hackerearth_bankloans/93fe009c-2-Foreclosure-Prediction-Dataset_/train_foreclosure.csv')

In [6]:
train['FORECLOSURE'].value_counts()

0    18217
1     1795
Name: FORECLOSURE, dtype: int64

In [137]:
test

,AGREEMENTID,FORECLOSURE
0,11220003,NaN
1,11220004,NaN
2,11220005,NaN
3,11220007,NaN
4,11220009,NaN
5,11220013,NaN
6,11220015,NaN
7,11220018,NaN
8,11220019,NaN
9,11220023,NaN


In [8]:
LMS[LMS['AGREEMENTID']==11220001][['AGREEMENTID','LOAN_AMT','CURRENT_ROI','EMI_DUEAMT','EMI_RECEIVED_AMT','OUTSTANDING_PRINCIPAL','PRE_EMI_DUEAMT','PAID_PRINCIPAL','PAID_INTEREST','LAST_RECEIPT_AMOUNT','LAST_RECEIPT_DATE','ORIGNAL_TENOR','COMPLETED_TENURE']]

,AGREEMENTID,LOAN_AMT,CURRENT_ROI,EMI_DUEAMT,EMI_RECEIVED_AMT,OUTSTANDING_PRINCIPAL,PRE_EMI_DUEAMT,PAID_PRINCIPAL,PAID_INTEREST,LAST_RECEIPT_AMOUNT,LAST_RECEIPT_DATE,ORIGNAL_TENOR,COMPLETED_TENURE
0,11220001,11710107.24,13.734072,7.516699e+06,7.332154e+06,2.262177e+06,7994.273589,9.447930e+06,2.617476e+06,83559.0,2013-11-01,120,39
2855,11220001,11710107.24,13.734072,7.701243e+06,7.516699e+06,2.157467e+06,7994.273589,9.552641e+06,2.638213e+06,83559.0,2013-12-01,120,40
5825,11220001,11710107.24,13.421934,7.885787e+06,7.701243e+06,2.051797e+06,7994.273589,9.658311e+06,2.657989e+06,83559.0,2014-01-01,120,41
8928,11220001,11710107.24,13.421934,7.885787e+06,7.885787e+06,1.944731e+06,7994.273589,9.765377e+06,2.676370e+06,83559.0,1974-02-02,120,41
12193,11220001,11710107.24,13.421934,8.254876e+06,8.070332e+06,1.836705e+06,7994.273589,9.873402e+06,2.693792e+06,83559.0,2014-03-01,120,43
15534,11220001,11710107.24,13.421934,8.439420e+06,8.254876e+06,1.727712e+06,7994.273589,9.982395e+06,2.710245e+06,83559.0,2014-04-01,120,44
18931,11220001,11710107.24,13.421934,8.614898e+06,8.439420e+06,1.164472e+05,7994.273589,1.159366e+07,2.725723e+06,1000000.0,2014-05-05,120,45


In [9]:
LMS[LMS['AGREEMENTID']==11220016][['LOAN_AMT','CURRENT_ROI','EMI_DUEAMT','OUTSTANDING_PRINCIPAL','PRE_EMI_DUEAMT','PAID_PRINCIPAL','PAID_INTEREST','LAST_RECEIPT_AMOUNT','ORIGNAL_TENOR','COMPLETED_TENURE']]

,LOAN_AMT,CURRENT_ROI,EMI_DUEAMT,OUTSTANDING_PRINCIPAL,PRE_EMI_DUEAMT,PAID_PRINCIPAL,PAID_INTEREST,LAST_RECEIPT_AMOUNT,ORIGNAL_TENOR,COMPLETED_TENURE
15,8407256.48,13.734072,3.650597e+06,5.031202e+06,185120.82864,3.376055e+06,1.848112e+06,50089.0,216,33
2868,8407256.48,13.734072,3.761221e+06,5.002122e+06,185120.82864,3.405134e+06,1.894231e+06,50089.0,216,34
5838,8407256.48,13.421934,3.871846e+06,4.972777e+06,185120.82864,3.434480e+06,1.940084e+06,50089.0,216,35
8941,8407256.48,13.421934,3.871846e+06,4.942126e+06,185120.82864,3.465130e+06,1.984632e+06,50089.0,216,35
12206,8407256.48,13.421934,4.093094e+06,4.911201e+06,185120.82864,3.496055e+06,2.028905e+06,50089.0,216,37
15547,8407256.48,13.421934,4.203718e+06,4.879999e+06,185120.82864,3.527258e+06,2.072901e+06,50089.0,216,38
18944,8407256.48,13.421934,4.314342e+06,4.848517e+06,185120.82864,3.558739e+06,2.116618e+06,50089.0,216,39
22626,8407256.48,13.421934,4.424966e+06,4.816753e+06,185120.82864,3.590503e+06,2.160052e+06,50089.0,216,40
26499,8407256.48,13.421934,4.535591e+06,4.784705e+06,185120.82864,3.622551e+06,2.203203e+06,50089.0,216,41
30383,8407256.48,13.421934,4.646215e+06,4.752370e+06,185120.82864,3.654886e+06,2.246066e+06,50089.0,216,42


In [10]:
LMS.columns

Index(['AGREEMENTID', 'CUSTOMERID', 'LOAN_AMT', 'NET_DISBURSED_AMT',
       'INTEREST_START_DATE', 'CURRENT_ROI', 'ORIGNAL_ROI', 'CURRENT_TENOR',
       'ORIGNAL_TENOR', 'DUEDAY', 'AUTHORIZATIONDATE', 'CITY',
       'PRE_EMI_DUEAMT', 'PRE_EMI_RECEIVED_AMT', 'PRE_EMI_OS_AMOUNT',
       'EMI_DUEAMT', 'EMI_RECEIVED_AMT', 'EMI_OS_AMOUNT', 'EXCESS_AVAILABLE',
       'EXCESS_ADJUSTED_AMT', 'BALANCE_EXCESS', 'NET_RECEIVABLE',
       'OUTSTANDING_PRINCIPAL', 'PAID_PRINCIPAL', 'PAID_INTEREST',
       'MONTHOPENING', 'LAST_RECEIPT_DATE', 'LAST_RECEIPT_AMOUNT', 'NET_LTV',
       'COMPLETED_TENURE', 'BALANCE_TENURE', 'DPD', 'FOIR', 'PRODUCT',
       'SCHEMEID', 'NPA_IN_LAST_MONTH', 'NPA_IN_CURRENT_MONTH', 'MOB'],
      dtype='object')

In [11]:
agreement_ids = LMS['AGREEMENTID'].unique()
len(agreement_ids)

33354

## Data Transformation 

In [12]:
COUNT = 0
Lms_cleaned = pd.DataFrame(columns=['AGREEMENTID','CUSTOMERID','LOAN_AMT','ROI','PRE_EMI','EMI','EMI_PAID','TOTAL_PAID','AVG_RECEIPT','TENURE','HL','LAP','STHL','STLAP'])
Lms_cleaned['AGREEMENTID'] = agreement_ids 
Lms_cleaned.set_index('AGREEMENTID', inplace = True)
for i in agreement_ids:
    Lms_i= LMS[LMS['AGREEMENTID']==i]
    Lms_cleaned.loc[i]['CUSTOMERID'] = Lms_i.iloc[0]['CUSTOMERID']
    Lms_cleaned.loc[i]['LOAN_AMT'] = Lms_i.iloc[0]['LOAN_AMT']
    Lms_cleaned.loc[i]['ROI'] = Lms_i.iloc[0]['CURRENT_ROI']
    Lms_cleaned.loc[i]['PRE_EMI'] = np.average(Lms_i['PRE_EMI_DUEAMT'])
    if len(Lms_i['EMI_DUEAMT'])>1:
        Lms_cleaned.loc[i]['EMI'] = Lms_i.iloc[1]['EMI_DUEAMT'] - Lms_i.iloc[0]['EMI_DUEAMT']
    else:
        if Lms_i.iloc[0]['COMPLETED_TENURE'] == 0:
             Lms_cleaned.loc[i]['EMI'] = Lms_i.iloc[0]['EMI_DUEAMT']
        else:        
             Lms_cleaned.loc[i]['EMI'] = Lms_i.iloc[0]['EMI_DUEAMT']/Lms_i.iloc[0]['COMPLETED_TENURE']
    Lms_cleaned.loc[i]['EMI_PAID'] = int(sum(Lms_i['EMI_DUEAMT'] - Lms_i['EMI_RECEIVED_AMT']) == 0)
    if len(Lms_i['EMI_DUEAMT'])>1:
        Lms_cleaned.loc[i]['TOTAL_PAID'] = (Lms_i.iloc[1]['PAID_INTEREST'] - Lms_i.iloc[0]['PAID_INTEREST'])+(Lms_i.iloc[1]['PAID_PRINCIPAL'] - Lms_i.iloc[0]['PAID_PRINCIPAL'])
    else:
        if Lms_i.iloc[0]['COMPLETED_TENURE'] == 0:
            Lms_cleaned.loc[i]['TOTAL_PAID'] = (Lms_i.iloc[0]['PAID_INTEREST'] +Lms_i.iloc[0]['PAID_PRINCIPAL'])
        else:
            Lms_cleaned.loc[i]['TOTAL_PAID'] = (Lms_i.iloc[0]['PAID_INTEREST'] +Lms_i.iloc[0]['PAID_PRINCIPAL'])/Lms_i.iloc[0]['COMPLETED_TENURE']
    #Lms_cleaned.loc[i]['AVG_RECEIPT'] = np.average(Lms_i['LAST_RECEIPT_AMOUNT'])
    Lms_cleaned.loc[i]['TENURE'] = Lms_i.iloc[0]['ORIGNAL_TENOR']
    Lms_cleaned.loc[i]['HL'] = int(Lms_i.iloc[0]['PRODUCT'] == 'HL')
    Lms_cleaned.loc[i]['LAP'] = int(Lms_i.iloc[0]['PRODUCT'] == 'LAP')
    Lms_cleaned.loc[i]['STHL'] = int(Lms_i.iloc[0]['PRODUCT'] == 'STHL')
    Lms_cleaned.loc[i]['STLAP'] = int(Lms_i.iloc[0]['PRODUCT'] == 'STLAP')
    COUNT +=1
    if COUNT%1000 ==0:
        print(COUNT)
    
Lms_cleaned

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000


,CUSTOMERID,LOAN_AMT,ROI,PRE_EMI,EMI,EMI_PAID,TOTAL_PAID,AVG_RECEIPT,TENURE,HL,LAP,STHL,STLAP
AGREEMENTID,,,,,,,,,,,,,
11220001,1.2001e+07,1.17101e+07,13.7341,7994.27,184544,0,125447,NaN,120,1,0,0,0
11220002,1.2001e+07,1.92903e+07,13.7341,100505,279448,0,312837,NaN,180,1,0,0,0
11220003,1.2001e+07,4.0535e+06,17.1676,0,68239.8,1,46387,NaN,180,1,0,0,0
11220004,1.2001e+07,1.3257e+07,15.6069,28441.7,194918,1,193884,NaN,180,1,0,0,0
11220005,1.2001e+07,1.32114e+07,16.5433,0,202948,1,137957,NaN,180,1,0,0,0
11220006,1.2001e+07,3.93339e+06,16.2312,5947.63,62983.5,1,42814,NaN,180,1,0,0,0
11220007,1.2001e+07,6.15531e+06,16.5433,451448,104347,1,70931.7,NaN,180,1,0,0,0
11220008,1.2001e+07,1.00226e+07,13.7341,60322.2,145192,0,98696.7,NaN,180,1,0,0,0
11220009,1.2001e+07,9.49928e+06,18.9156,188502,178778,0,131790,NaN,180,0,1,0,0


In [13]:
ld = Lms_cleaned.drop(['AVG_RECEIPT'],axis = 1)
Lms_cleaned = ld

In [14]:
Lms_cleaned_nonan = Lms_cleaned.dropna()

In [15]:
Lms_cleaned_nonan

,CUSTOMERID,LOAN_AMT,ROI,PRE_EMI,EMI,EMI_PAID,TOTAL_PAID,TENURE,HL,LAP,STHL,STLAP
AGREEMENTID,,,,,,,,,,,,
11220001,1.2001e+07,1.17101e+07,13.7341,7994.27,184544,0,125447,120,1,0,0,0
11220002,1.2001e+07,1.92903e+07,13.7341,100505,279448,0,312837,180,1,0,0,0
11220003,1.2001e+07,4.0535e+06,17.1676,0,68239.8,1,46387,180,1,0,0,0
11220004,1.2001e+07,1.3257e+07,15.6069,28441.7,194918,1,193884,180,1,0,0,0
11220005,1.2001e+07,1.32114e+07,16.5433,0,202948,1,137957,180,1,0,0,0
11220006,1.2001e+07,3.93339e+06,16.2312,5947.63,62983.5,1,42814,180,1,0,0,0
11220007,1.2001e+07,6.15531e+06,16.5433,451448,104347,1,70931.7,180,1,0,0,0
11220008,1.2001e+07,1.00226e+07,13.7341,60322.2,145192,0,98696.7,180,1,0,0,0
11220009,1.2001e+07,9.49928e+06,18.9156,188502,178778,0,131790,180,0,1,0,0


In [16]:
len(Lms_cleaned) - len(Lms_cleaned_nonan)

459

### Joining LMS_cleaned with train

In [17]:
Lms_clean_joined = pd.merge(Lms_cleaned_nonan,train,left_index = True, right_on = "AGREEMENTID")
Lms_clean_joined

,CUSTOMERID,LOAN_AMT,ROI,PRE_EMI,EMI,EMI_PAID,TOTAL_PAID,TENURE,HL,LAP,STHL,STLAP,AGREEMENTID,FORECLOSURE
0,1.2001e+07,1.17101e+07,13.7341,7994.27,184544,0,125447,120,1,0,0,0,11220001,1
1,1.2001e+07,1.92903e+07,13.7341,100505,279448,0,312837,180,1,0,0,0,11220002,1
2,1.2001e+07,3.93339e+06,16.2312,5947.63,62983.5,1,42814,180,1,0,0,0,11220006,1
3,1.2001e+07,1.00226e+07,13.7341,60322.2,145192,0,98696.7,180,1,0,0,0,11220008,1
4,1.2001e+07,7.75594e+06,15.6069,27732.8,120218,1,81720,180,1,0,0,0,11220010,1
5,1.2001e+07,954408,13.7341,1325.13,13827.7,0,9399.61,180,1,0,0,0,11220011,0
6,1.2001e+07,6.6057e+06,15.6069,37790.5,95533.1,0,64940.1,300,1,0,0,0,11220012,0
7,1.2001e+07,3.29603e+07,13.7341,225933,464308,0,315620,300,1,0,0,0,11220014,1
8,1.2001e+07,8.40726e+06,13.7341,185121,110624,0,75198.4,216,1,0,0,0,11220016,1
9,1.2001e+07,975842,17.4797,1674.08,16756.3,0,11390.3,180,0,1,0,0,11220017,0


In [18]:
Lms_cleaned[Lms_cleaned.isnull().values]

,CUSTOMERID,LOAN_AMT,ROI,PRE_EMI,EMI,EMI_PAID,TOTAL_PAID,TENURE,HL,LAP,STHL,STLAP
AGREEMENTID,,,,,,,,,,,,
11252411,NaN,1.41143e+06,17.4797,2019.72,0,1,2745.87,240,0,0,1,0
11252412,NaN,2.7818e+06,13.7341,1252.25,46514.3,0,851.235,180,0,0,0,1
11252718,NaN,1.20104e+06,13.7341,1350.53,0,1,1836.08,144,0,1,0,0
11252719,NaN,3.72754e+06,13.4219,9825.85,0,1,13358.5,240,0,0,1,0
11252720,NaN,1.5013e+06,16.2312,1994.32,0,1,2711.34,240,0,0,1,0
11252721,NaN,1.60639e+06,14.9826,1688.44,0,1,2295.48,144,0,0,1,0
11252722,NaN,3.85233e+06,13.4219,12692.5,0,1,17255.9,240,0,0,1,0
11252723,NaN,1.65143e+06,14.6705,1983.28,0,1,2696.33,192,0,0,1,0
11252724,NaN,3.75324e+06,17.4173,5349.11,0,1,7272.28,240,0,0,1,0


In [19]:
Lms_cleaned.dropna()

,CUSTOMERID,LOAN_AMT,ROI,PRE_EMI,EMI,EMI_PAID,TOTAL_PAID,TENURE,HL,LAP,STHL,STLAP
AGREEMENTID,,,,,,,,,,,,
11220001,1.2001e+07,1.17101e+07,13.7341,7994.27,184544,0,125447,120,1,0,0,0
11220002,1.2001e+07,1.92903e+07,13.7341,100505,279448,0,312837,180,1,0,0,0
11220003,1.2001e+07,4.0535e+06,17.1676,0,68239.8,1,46387,180,1,0,0,0
11220004,1.2001e+07,1.3257e+07,15.6069,28441.7,194918,1,193884,180,1,0,0,0
11220005,1.2001e+07,1.32114e+07,16.5433,0,202948,1,137957,180,1,0,0,0
11220006,1.2001e+07,3.93339e+06,16.2312,5947.63,62983.5,1,42814,180,1,0,0,0
11220007,1.2001e+07,6.15531e+06,16.5433,451448,104347,1,70931.7,180,1,0,0,0
11220008,1.2001e+07,1.00226e+07,13.7341,60322.2,145192,0,98696.7,180,1,0,0,0
11220009,1.2001e+07,9.49928e+06,18.9156,188502,178778,0,131790,180,0,1,0,0


In [20]:
labels_only_lms = Lms_clean_joined['FORECLOSURE']
Lms_clean_joined = Lms_clean_joined.drop(['FORECLOSURE'],axis = 1)

In [21]:
Lms_cleaned_nonan[Lms_cleaned_nonan.isnull().values]
Lms_cleaned_nonan = Lms_cleaned_nonan.dropna()

In [22]:
Lms_clean_joined['CUSTOMERID'] = Lms_clean_joined['CUSTOMERID'].astype(np.int64)
type(Lms_clean_joined['CUSTOMERID'][0])

numpy.int64

In [23]:
Lms_cleaned_nonan['CUSTOMERID'] = Lms_cleaned_nonan['CUSTOMERID'].astype(np.int64)
Lms_cleaned_nonan.reset_index(inplace = True)
type(Lms_cleaned_nonan['CUSTOMERID'][0])
Lms_cleaned_nonan['CUSTOMERID']


0        12001000
1        12001001
2        12001002
3        12001003
4        12001004
5        12001005
6        12001006
7        12001007
8        12001008
9        12001009
10       12001010
11       12001011
12       12001012
13       12001013
14       12001014
15       12001015
16       12001016
17       12001017
18       12001018
19       12001019
20       12001020
21       12001021
22       12001022
23       12001023
24       12001024
25       12001025
26       12001026
27       12001027
28       12001028
29       12001029
           ...   
32865    12033866
32866    12033867
32867    12033868
32868    12033869
32869    12033870
32870    12033871
32871    12033872
32872    12033873
32873    12033874
32874    12033875
32875    12033876
32876    12033877
32877    12033878
32878    12033879
32879    12033880
32880    12033881
32881    12033882
32882    12033883
32883    12033884
32884    12033885
32885    12033886
32886    12033887
32887    12033888
32888    12033889
32889    1

In [ ]:
plt.plot([])

In [24]:
lms_test = Lms_cleaned.dropna()
lms_test

,CUSTOMERID,LOAN_AMT,ROI,PRE_EMI,EMI,EMI_PAID,TOTAL_PAID,TENURE,HL,LAP,STHL,STLAP
AGREEMENTID,,,,,,,,,,,,
11220001,1.2001e+07,1.17101e+07,13.7341,7994.27,184544,0,125447,120,1,0,0,0
11220002,1.2001e+07,1.92903e+07,13.7341,100505,279448,0,312837,180,1,0,0,0
11220003,1.2001e+07,4.0535e+06,17.1676,0,68239.8,1,46387,180,1,0,0,0
11220004,1.2001e+07,1.3257e+07,15.6069,28441.7,194918,1,193884,180,1,0,0,0
11220005,1.2001e+07,1.32114e+07,16.5433,0,202948,1,137957,180,1,0,0,0
11220006,1.2001e+07,3.93339e+06,16.2312,5947.63,62983.5,1,42814,180,1,0,0,0
11220007,1.2001e+07,6.15531e+06,16.5433,451448,104347,1,70931.7,180,1,0,0,0
11220008,1.2001e+07,1.00226e+07,13.7341,60322.2,145192,0,98696.7,180,1,0,0,0
11220009,1.2001e+07,9.49928e+06,18.9156,188502,178778,0,131790,180,0,1,0,0


In [25]:
lms_test['CUSTOMERID'] = lms_test['CUSTOMERID'].astype(np.int64)
lms_test.reset_index(inplace=True)
type(lms_test['CUSTOMERID'][0])

/home/soham/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


numpy.int64

In [26]:
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

## Oversampling using SMOTE

In [27]:
X_train, X_test, y_train, y_test = train_test_split(Lms_clean_joined, labels_only_lms, test_size=0.33, random_state=42)

In [28]:
y_train.value_counts()

0    12022
1     1197
Name: FORECLOSURE, dtype: int64

In [29]:
sm = SMOTE(random_state=12, ratio = 1.0)
x_train_res, y_train_res = sm.fit_sample(X_train, y_train)

In [31]:
pd.Series(y_train_res).value_counts()

1    12022
0    12022
dtype: int64

In [121]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import recall_score,precision_score


In [58]:
clf_rf = RandomForestClassifier(n_estimators=25, random_state=12)
clf_rf.fit(x_train_res, y_train_res)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=25, n_jobs=None,
            oob_score=False, random_state=12, verbose=0, warm_start=False)

In [123]:
print('Train results')
print(clf_rf.score(X_train, y_train))
print(recall_score(y_train, clf_rf.predict(X_train)))
print(precision_score(y_train, clf_rf.predict(X_train)))
print('Validation Results')
print(clf_rf.score(X_test, y_test))
print(recall_score(y_test, clf_rf.predict(X_test)))
print(precision_score(y_test, clf_rf.predict(X_test)))

Train results
0.9990922157500567
0.9949874686716792
0.9949874686716792
Validation Results
0.9204545454545454
0.697324414715719
0.553050397877984


In [38]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

In [41]:
y_pred = clf_rf.predict(X_test)

In [43]:
conf_mat = confusion_matrix(y_test, y_pred)
print(conf_mat)

[[5577  337]
 [ 181  417]]


In [103]:
y_pred_proba = clf_rf.predict_proba(X_test)
len(y_pred_proba)

6512

In [104]:
k = y_pred_proba
sum= 0
for i in range(len(k)):
    if k[i][0] < 0.85:
        print(k[i][0])


0.72
0.0
0.52
0.44
0.84
0.08
0.8
0.6
0.6
0.64
0.52
0.32
0.24
0.2
0.0
0.24
0.68
0.6
0.64
0.8
0.28
0.44
0.0
0.36
0.72
0.52
0.68
0.72
0.6
0.04
0.44
0.64
0.8
0.4
0.24
0.76
0.68
0.72
0.6
0.48
0.8
0.84
0.4
0.4
0.84
0.24
0.04
0.32
0.04
0.8
0.6
0.56
0.36
0.64
0.76
0.0
0.2
0.44
0.08
0.16
0.24
0.36
0.56
0.48
0.2
0.2
0.72
0.32
0.84
0.16
0.32
0.28
0.6
0.44
0.08
0.28
0.36
0.08
0.72
0.08
0.0
0.16
0.8
0.12
0.12
0.68
0.08
0.44
0.12
0.84
0.68
0.44
0.4
0.44
0.68
0.24
0.2
0.12
0.84
0.6
0.68
0.76
0.84
0.68
0.04
0.72
0.24
0.16
0.08
0.44
0.24
0.36
0.28
0.56
0.8
0.16
0.68
0.76
0.8
0.4
0.28
0.72
0.52
0.72
0.64
0.0
0.48
0.36
0.36
0.84
0.56
0.84
0.72
0.56
0.72
0.04
0.2
0.68
0.6
0.04
0.12
0.68
0.68
0.72
0.68
0.84
0.64
0.52
0.04
0.2
0.64
0.76
0.04
0.0
0.68
0.6
0.44
0.12
0.8
0.36
0.8
0.84
0.8
0.24
0.4
0.36
0.0
0.08
0.44
0.4
0.04
0.64
0.2
0.0
0.48
0.64
0.4
0.84
0.64
0.0
0.24
0.48
0.28
0.24
0.08
0.32
0.12
0.4
0.64
0.72
0.0
0.24
0.2
0.72
0.12
0.48
0.36
0.24
0.24
0.0
0.2
0.64
0.72
0.56
0.36
0.0
0.64
0.68
0.52
0.64
0.8

In [45]:
417/(417+181)

0.697324414715719

## Submission 

In [107]:
Lms_clean_joined_sub = Lms_clean_joined.drop(['AGREEMENTID','CUSTOMERID'],axis = 1)

In [109]:
sm_sub = SMOTE(random_state=12, ratio = 1.0)
x_train_res_sub, y_train_res_sub = sm_sub.fit_sample(Lms_clean_joined_sub,labels_only_lms)

In [111]:
clf_rf_sub = RandomForestClassifier(n_estimators=25, random_state=12)
clf_rf_sub.fit(x_train_res_sub, y_train_res_sub)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=25, n_jobs=None,
            oob_score=False, random_state=12, verbose=0, warm_start=False)

In [124]:
print('Train results')
print(clf_rf_sub.score(Lms_clean_joined_sub, labels_only_lms))
print(recall_score(labels_only_lms, clf_rf_sub.predict(Lms_clean_joined_sub)))
print(precision_score(labels_only_lms, clf_rf_sub.predict(Lms_clean_joined_sub)))

Train results
0.9991384116365111
0.9977715877437325
0.9927937915742794


In [126]:
lms_cleaned_predict_sub = pd.merge(Lms_cleaned,test,left_index = True, right_on='AGREEMENTID' )
lms_cleaned_predict_sub.drop(['CUSTOMERID','AGREEMENTID','FORECLOSURE'], inplace = True, axis = 1)
lms_cleaned_predict_sub

,LOAN_AMT,ROI,PRE_EMI,EMI,EMI_PAID,TOTAL_PAID,TENURE,HL,LAP,STHL,STLAP
0,4.0535e+06,17.1676,0,68239.8,1,46387,180,1,0,0,0
1,1.3257e+07,15.6069,28441.7,194918,1,193884,180,1,0,0,0
2,1.32114e+07,16.5433,0,202948,1,137957,180,1,0,0,0
3,6.15531e+06,16.5433,451448,104347,1,70931.7,180,1,0,0,0
4,9.49928e+06,18.9156,188502,178778,0,131790,180,0,1,0,0
5,5.40066e+06,13.7341,5627.39,75269.7,0,51165.7,240,1,0,0,0
6,4.30872e+06,17.4797,24033.5,73971,0,50282.9,180,0,1,0,0
7,7.68864e+06,16.5433,0,118111,1,80287.8,180,1,0,0,0
8,1.7252e+07,13.7341,167786,269322,0,183076,156,1,0,0,0
9,1.13752e+07,19.0404,20453.4,230513,1,156695,120,0,1,0,0


In [134]:
lms_cleaned_predict_sub.isnull().values.any()

False

In [135]:
y_pred_proba_sub = clf_rf_sub.predict_proba(lms_cleaned_predict_sub)

In [136]:
y_pred_proba_sub.shape

(13342, 2)

In [148]:
test_pred = test.copy()
test_pred['FORECLOSURE'] = y_pred_proba_sub[:,1]
#test_pred.drop(['FORECLOSURE'],axis = 1,inplace = True)



In [151]:
test_pred
test_pred.to_csv('/home/soham/hackerearth_bankloans/result.csv',index=False)

## Penalized RF without SMOTE

In [160]:
Lms_clean_joined

,CUSTOMERID,LOAN_AMT,ROI,PRE_EMI,EMI,EMI_PAID,TOTAL_PAID,TENURE,HL,LAP,STHL,STLAP,AGREEMENTID
0,12001000,1.17101e+07,13.7341,7994.27,184544,0,125447,120,1,0,0,0,11220001
1,12001001,1.92903e+07,13.7341,100505,279448,0,312837,180,1,0,0,0,11220002
2,12001005,3.93339e+06,16.2312,5947.63,62983.5,1,42814,180,1,0,0,0,11220006
3,12001007,1.00226e+07,13.7341,60322.2,145192,0,98696.7,180,1,0,0,0,11220008
4,12001009,7.75594e+06,15.6069,27732.8,120218,1,81720,180,1,0,0,0,11220010
5,12001010,954408,13.7341,1325.13,13827.7,0,9399.61,180,1,0,0,0,11220011
6,12001011,6.6057e+06,15.6069,37790.5,95533.1,0,64940.1,300,1,0,0,0,11220012
7,12001013,3.29603e+07,13.7341,225933,464308,0,315620,300,1,0,0,0,11220014
8,12001015,8.40726e+06,13.7341,185121,110624,0,75198.4,216,1,0,0,0,11220016
9,12001016,975842,17.4797,1674.08,16756.3,0,11390.3,180,0,1,0,0,11220017


In [162]:
X_train_wrf, X_test_wrf, y_train_wrf, y_test_wrf = train_test_split(Lms_clean_joined, labels_only_lms, test_size=0.33, random_state=42)

In [165]:
y_train_wrf.value_counts()

0    12022
1     1197
Name: FORECLOSURE, dtype: int64

In [189]:
max_w = 0 
w_max = 0
for w in range(500):
    if w%50 ==0:
        print(w)
    RF = RandomForestClassifier(class_weight={0: 1, 1: w},n_estimators=25,random_state=14)
    RF.fit(X_train_wrf, y_train_wrf)
    max_recall1 = recall_score(y_test_wrf, RF.predict(X_test_wrf))
    max_precision1 = precision_score(y_train_wrf, RF.predict(X_train_wrf))
    #max_accuracy1 = RF.score(X_train_wrf, y_train_wrf)
    max_w1 = 2*max_recall1*max_precision1/(max_recall1+max_precision1)
    if max_w1 > max_w:
        max_w = max_w1
        w_max = w
print(max_w,w_max)        

0


/home/soham/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/soham/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


50
100
150
200
250
300
350
400
450
0.6867594037983961 5


In [184]:
RF = RandomForestClassifier(class_weight={0: 1, 1: 5},n_estimators=25,random_state=14)
RF.fit(X_train_wrf, y_train_wrf)

RandomForestClassifier(bootstrap=True, class_weight={0: 1, 1: 5},
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=25, n_jobs=None, oob_score=False, random_state=14,
            verbose=0, warm_start=False)

In [185]:
print('Train results')
print(RF.score(X_train_wrf, y_train_wrf))
print(recall_score(y_train_wrf, RF.predict(X_train_wrf)))
print(precision_score(y_train_wrf, RF.predict(X_train_wrf)))
print('Validation Results')
print(RF.score(X_test_wrf, y_test_wrf))
print(recall_score(y_test_wrf, RF.predict(X_test_wrf)))
print(precision_score(y_test_wrf, RF.predict(X_test_wrf)))

Train results
0.9990922157500567
0.9916457811194653
0.9983179142136249
Validation Results
0.9315110565110565
0.5234113712374582
0.6603375527426161


## Submission of Penalized RF without SMOTE

In [187]:
RF_sub = RandomForestClassifier(class_weight={0: 1, 1: 5},n_estimators=25,random_state=14)
RF_sub.fit(Lms_clean_joined_sub, labels_only_lms)

RandomForestClassifier(bootstrap=True, class_weight={0: 1, 1: 5},
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=25, n_jobs=None, oob_score=False, random_state=14,
            verbose=0, warm_start=False)

In [188]:
print('Train results')
print(RF_sub.score(Lms_clean_joined_sub, labels_only_lms))
print(recall_score(labels_only_lms, RF_sub.predict(Lms_clean_joined_sub)))
print(precision_score(labels_only_lms, RF_sub.predict(Lms_clean_joined_sub)))

Train results
0.9982768232730221
0.9844011142061281
0.9966159052453468


In [190]:
y_pred_proba_wrf_sub = RF_sub.predict_proba(lms_cleaned_predict_sub)

In [194]:
test_pred_wrf = test.copy()
test_pred_wrf['FORECLOSURE'] = y_pred_proba_wrf_sub[:,1]
#test_pred.drop(['FORECLOSURE'],axis = 1,inplace = True)



In [195]:
test_pred_wrf.to_csv('/home/soham/hackerearth_bankloans/result_wrf.csv',index=False)

## Demographics

In [152]:
dg =  pd.read_excel('/home/soham/hackerearth_bankloans/93fe009c-2-Foreclosure-Prediction-Dataset_/Customers_31JAN2019.xlsx');

In [153]:
custtypeid_oh = pd.get_dummies(dg['CUST_CONSTTYPE_ID'])
custtypeid_oh.columns = ['CUST_CONSTTYPE_ID_1','CUST_CONSTTYPE_ID_2','CUST_CONSTTYPE_ID_3','CUST_CONSTTYPE_ID_4','CUST_CONSTTYPE_ID_5','CUST_CONSTTYPE_ID_6']
dg1 = pd.concat([dg,custtypeid_oh],axis = 1)
custcatid_oh = pd.get_dummies(dg['CUST_CATEGORYID'])
custcatid_oh.columns = ['CUST_CATEGORYID_4','CUST_CATEGORYID_5','CUST_CATEGORYID_6','CUST_CATEGORYID_7','CUST_CATEGORYID_8']
dg1 = pd.concat([dg1,custcatid_oh],axis = 1)
sex = pd.get_dummies(dg['SEX'])
sex.columns = ['FEMALE','MALE']
married = pd.get_dummies(dg['MARITAL_STATUS'])
married.columns = ['MARRIED','SINGLE']
qual = pd.get_dummies(dg['QUALIFICATION'])
dg1 = pd.concat([dg1,sex,married,qual],axis = 1)

In [154]:
dg2 = dg1.drop(['CUST_CONSTTYPE_ID','CUST_CATEGORYID','PROFESSION','SEX','MARITAL_STATUS','QUALIFICATION','OCCUPATION','POSITION'],axis = 1)
dg2['PRE_JOBYEARS'] = dg2['PRE_JOBYEARS'].fillna(0)
dg2

,CUSTOMERID,AGE,NO_OF_DEPENDENT,GROSS_INCOME,PRE_JOBYEARS,NETTAKEHOMEINCOME,BRANCH_PINCODE,CUST_CONSTTYPE_ID_1,CUST_CONSTTYPE_ID_2,CUST_CONSTTYPE_ID_3,...,MARRIED,SINGLE,DOCTRATE,ENG,GRAD,MAT,OTHERS,POSTGRAD,PRF,UG
0,12001000,33.0,0.0,1.983752e+05,8.0,1.983752e+05,400070.0,1,0,0,...,1,0,0,0,0,0,0,1,0,0
1,12001001,44.0,0.0,2.427040e+05,10.0,2.427040e+05,400070.0,1,0,0,...,1,0,0,0,0,0,0,1,0,0
2,12001002,50.0,0.0,0.000000e+00,0.0,0.000000e+00,400070.0,1,0,0,...,1,0,0,0,1,0,0,0,0,0
3,12001003,40.0,0.0,3.652635e+05,0.0,3.652635e+05,400070.0,0,0,1,...,1,0,0,0,1,0,0,0,0,0
4,12001004,27.0,0.0,1.873422e+05,0.0,1.873422e+05,400070.0,0,0,1,...,1,0,0,0,1,0,0,0,0,0
5,12001005,43.0,0.0,5.353621e+04,0.0,5.353621e+04,400070.0,0,0,1,...,1,0,0,0,1,0,0,0,0,0
6,12001006,48.0,0.0,4.984215e+04,0.0,4.984215e+04,400070.0,0,0,1,...,1,0,0,0,0,0,0,0,0,1
7,12001007,33.0,0.0,1.233705e+05,7.0,1.233705e+05,400070.0,1,0,0,...,1,0,0,0,0,0,0,1,0,0
8,12001008,44.0,0.0,1.833320e+05,0.0,1.833320e+05,400070.0,1,0,0,...,1,0,0,0,1,0,0,0,0,0
9,12001009,37.0,0.0,9.380384e+04,0.0,9.380384e+04,400070.0,0,0,1,...,1,0,0,0,0,0,0,0,0,1


In [155]:
dg2[dg2.isnull().values]

,CUSTOMERID,AGE,NO_OF_DEPENDENT,GROSS_INCOME,PRE_JOBYEARS,NETTAKEHOMEINCOME,BRANCH_PINCODE,CUST_CONSTTYPE_ID_1,CUST_CONSTTYPE_ID_2,CUST_CONSTTYPE_ID_3,...,MARRIED,SINGLE,DOCTRATE,ENG,GRAD,MAT,OTHERS,POSTGRAD,PRF,UG
37,12001039,NaN,0.0,0.000000,0.0,0.000000,400070.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
63,12001070,NaN,0.0,0.000000,0.0,0.000000,400070.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
71,12001082,NaN,0.0,0.000000,0.0,0.000000,400070.0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
72,12001086,NaN,0.0,0.000000,0.0,0.000000,400070.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
73,12001087,NaN,0.0,0.000000,0.0,0.000000,400070.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
74,12001088,NaN,NaN,0.000000,0.0,0.000000,NaN,0,1,0,...,0,0,0,0,0,0,0,0,0,0
74,12001088,NaN,NaN,0.000000,0.0,0.000000,NaN,0,1,0,...,0,0,0,0,0,0,0,0,0,0
74,12001088,NaN,NaN,0.000000,0.0,0.000000,NaN,0,1,0,...,0,0,0,0,0,0,0,0,0,0
82,12001099,NaN,0.0,0.000000,0.0,0.000000,110001.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
87,12001106,NaN,0.0,0.000000,0.0,0.000000,400070.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [156]:
dg2['NO_OF_DEPENDENT'] = dg2['NO_OF_DEPENDENT'].fillna(0)
dg2['AGE'] = dg2['AGE'].fillna(dg2['AGE'].mean())

In [157]:
len(dg2)

10000

### Joining LMS with demographics

In [158]:
lms_dg2 = pd.merge(Lms_clean_joined,dg2)
len(lms_dg2)

5992

In [159]:
lms_dg4 = pd.merge(lms_test,dg2)

In [112]:
lms_dg2['FORECLOSURE'].value_counts()

0    4561
1    1431
Name: FORECLOSURE, dtype: int64

In [24]:
dg2 = dg2.drop(['BRANCH_PINCODE'],axis = 1)
dg2.head()

,CUSTOMERID,AGE,NO_OF_DEPENDENT,GROSS_INCOME,PRE_JOBYEARS,NETTAKEHOMEINCOME,CUST_CONSTTYPE_ID_1,CUST_CONSTTYPE_ID_2,CUST_CONSTTYPE_ID_3,CUST_CONSTTYPE_ID_4,...,MARRIED,SINGLE,DOCTRATE,ENG,GRAD,MAT,OTHERS,POSTGRAD,PRF,UG
0,12001000,33.0,0.0,198375.221829,8.0,198375.221829,1,0,0,0,...,1,0,0,0,0,0,0,1,0,0
1,12001001,44.0,0.0,242703.982915,10.0,242703.982915,1,0,0,0,...,1,0,0,0,0,0,0,1,0,0
2,12001002,50.0,0.0,0.000000,0.0,0.000000,1,0,0,0,...,1,0,0,0,1,0,0,0,0,0
3,12001003,40.0,0.0,365263.511624,0.0,365263.516128,0,0,1,0,...,1,0,0,0,1,0,0,0,0,0
4,12001004,27.0,0.0,187342.198995,0.0,187342.198995,0,0,1,0,...,1,0,0,0,1,0,0,0,0,0


In [25]:
dg2.isnull().values.any()

False

### Training just on LMS_cleaned

In [17]:
Lms_clean_joined.drop(['CUSTOMERID','AGREEMENTID'],inplace = True,axis = 1)

In [18]:
Lms_clean_joined.shape

(19731, 11)

In [80]:
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation
import keras.backend as K
def custom_activation(x):
    print(x)
    return K.sigmoid(x)

get_custom_objects().update({'custom_activation': Activation(custom_activation)})


In [83]:
model1 = Sequential()
model1.add(Dense(32, activation='relu', input_dim=11))
model1.add(Dense(16, activation='relu'))
model1.add(Dense(1, activation='sigmoid'))

In [71]:
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 32)                448       
_________________________________________________________________
dense_7 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 17        
Total params: 993
Trainable params: 993
Non-trainable params: 0
_________________________________________________________________


In [72]:
model1.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy'])

In [73]:
#x_train_res_dropped = x_train_res.drop(['AGREEMENTID','CUSTOMERID'])

In [74]:
model1.fit(x_train_res, y_train_res, epochs = 10, batch_size = 64)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
24044/24044 [==============================] - 6s 252us/step - loss: 0.4616 - acc: 0.5384
Epoch 2/10
24044/24044 [==============================] - 1s 22us/step - loss: 0.4616 - acc: 0.5384
Epoch 3/10
24044/24044 [==============================] - 1s 21us/step - loss: 0.4616 - acc: 0.5384
Epoch 4/10
24044/24044 [==============================] - 1s 21us/step - loss: 0.4616 - acc: 0.5384
Epoch 5/10
24044/24044 [==============================] - 1s 22us/step - loss: 0.4616 - acc: 0.5384
Epoch 6/10
24044/24044 [==============================] - 1s 22us/step - loss: 0.4616 - acc: 0.5384
Epoch 7/10
24044/24044 [==============================] - 1s 22us/step - loss: 0.4616 - acc: 0.5384
Epoch 8/10
24044/24044 [==============================] - 1s 22us/step - loss: 0.4616 - acc: 0.5384
Epoch 9/10
24044/24044 [==============================] - 1s 22us/step - loss: 0.4616 - acc: 0.5384
Epoch 10/10
24044/24044 [==========================

In [80]:
lms_cleaned_predict = pd.merge(Lms_cleaned_nonan,test )
lms_cleaned_predict.drop(['CUSTOMERID','AGREEMENTID','FORECLOSURE'], inplace = True, axis = 1)
lms_cleaned_predict

,LOAN_AMT,ROI,PRE_EMI,EMI,EMI_PAID,TOTAL_PAID,TENURE,HL,LAP,STHL,STLAP
0,4.0535e+06,17.1676,0,68239.8,1,46387,180,1,0,0,0
1,1.3257e+07,15.6069,28441.7,194918,1,193884,180,1,0,0,0
2,1.32114e+07,16.5433,0,202948,1,137957,180,1,0,0,0
3,6.15531e+06,16.5433,451448,104347,1,70931.7,180,1,0,0,0
4,9.49928e+06,18.9156,188502,178778,0,131790,180,0,1,0,0
5,5.40066e+06,13.7341,5627.39,75269.7,0,51165.7,240,1,0,0,0
6,4.30872e+06,17.4797,24033.5,73971,0,50282.9,180,0,1,0,0
7,7.68864e+06,16.5433,0,118111,1,80287.8,180,1,0,0,0
8,1.7252e+07,13.7341,167786,269322,0,183076,156,1,0,0,0
9,1.13752e+07,19.0404,20453.4,230513,1,156695,120,0,1,0,0


In [84]:
predictions = model1.predict(lms_cleaned_predict)

In [94]:
k = predictions
sum= 0
for i in range(len(k)):
    sum = sum + k[i][0]
sum

0.0

### Training on LMS with demographics

In [201]:
lms_dg3 = pd.merge(lms_dg2,train)
lms_dg3_pred = lms_dg3
lms_dg3_pred['AGREEMENTID']
lms_dg3.drop(['CUSTOMERID','AGREEMENTID'],inplace = True,axis=1)
lms_dg3


,LOAN_AMT,ROI,PRE_EMI,EMI,EMI_PAID,TOTAL_PAID,TENURE,HL,LAP,STHL,...,MARRIED,SINGLE,DOCTRATE,ENG,GRAD,MAT,OTHERS,POSTGRAD,PRF,UG
0,1.17101e+07,13.7341,7994.27,184544,0,125447,120,1,0,0,...,1,0,0,0,0,0,0,1,0,0
1,1.92903e+07,13.7341,100505,279448,0,312837,180,1,0,0,...,1,0,0,0,0,0,0,1,0,0
2,3.93339e+06,16.2312,5947.63,62983.5,1,42814,180,1,0,0,...,1,0,0,0,1,0,0,0,0,0
3,1.00226e+07,13.7341,60322.2,145192,0,98696.7,180,1,0,0,...,1,0,0,0,0,0,0,1,0,0
4,7.75594e+06,15.6069,27732.8,120218,1,81720,180,1,0,0,...,1,0,0,0,0,0,0,0,0,1
5,954408,13.7341,1325.13,13827.7,0,9399.61,180,1,0,0,...,1,0,0,0,1,0,0,0,0,0
6,6.6057e+06,15.6069,37790.5,95533.1,0,64940.1,300,1,0,0,...,1,0,0,0,1,0,0,0,0,0
7,3.29603e+07,13.7341,225933,464308,0,315620,300,1,0,0,...,1,0,0,0,0,0,0,1,0,0
8,8.40726e+06,13.7341,185121,110624,0,75198.4,216,1,0,0,...,1,0,0,0,1,0,0,0,0,0
9,1.46649e+07,15.6069,141577,215617,1,146569,180,1,0,0,...,1,0,0,0,1,0,0,0,0,0


In [202]:
lms_dg3_labels = lms_dg3['FORECLOSURE']
lms_dg3.drop(['FORECLOSURE'],axis = 1,inplace=True)


In [203]:
lms_dg3.drop(['BRANCH_PINCODE'],axis = 1,inplace = True)

In [204]:
lms_dg3.shape

(5992, 39)

In [251]:
model = Sequential()
# Dense(64) is a fully-connected layer with 64 hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 20-dimensional vectors.
model.add(Dense(32, activation='relu', input_dim=39))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [252]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [253]:
model.fit(lms_dg3, lms_dg3_labels, epochs = 3,validation_split=0.2, batch_size = 32)

Train on 4793 samples, validate on 1199 samples
Epoch 1/3
4793/4793 [==============================] - 1s 224us/step - loss: 4.6406 - acc: 0.7121 - val_loss: 0.7259 - val_acc: 0.9550
Epoch 2/3
4793/4793 [==============================] - 0s 71us/step - loss: 4.6406 - acc: 0.7121 - val_loss: 0.7259 - val_acc: 0.9550
Epoch 3/3
4793/4793 [==============================] - 0s 69us/step - loss: 4.6406 - acc: 0.7121 - val_loss: 0.7259 - val_acc: 0.9550


In [138]:
lms_dg3_pred.columns == 'AGREEMENTID'

array([False, False, False, False, False, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False])

In [140]:
test.columns

Index(['AGREEMENTID', 'FORECLOSURE'], dtype='object')

In [181]:
lms_dg4

,AGREEMENTID,CUSTOMERID,LOAN_AMT,ROI,PRE_EMI,EMI,EMI_PAID,TOTAL_PAID,TENURE,HL,...,MARRIED,SINGLE,DOCTRATE,ENG,GRAD,MAT,OTHERS,POSTGRAD,PRF,UG
0,11220001,12001000,1.17101e+07,13.7341,7994.27,184544,0,125447,120,1,...,1,0,0,0,0,0,0,1,0,0
1,11220002,12001001,1.92903e+07,13.7341,100505,279448,0,312837,180,1,...,1,0,0,0,0,0,0,1,0,0
2,11220003,12001002,4.0535e+06,17.1676,0,68239.8,1,46387,180,1,...,1,0,0,0,1,0,0,0,0,0
3,11220004,12001003,1.3257e+07,15.6069,28441.7,194918,1,193884,180,1,...,1,0,0,0,1,0,0,0,0,0
4,11220005,12001004,1.32114e+07,16.5433,0,202948,1,137957,180,1,...,1,0,0,0,1,0,0,0,0,0
5,11220006,12001005,3.93339e+06,16.2312,5947.63,62983.5,1,42814,180,1,...,1,0,0,0,1,0,0,0,0,0
6,11220007,12001006,6.15531e+06,16.5433,451448,104347,1,70931.7,180,1,...,1,0,0,0,0,0,0,0,0,1
7,11220008,12001007,1.00226e+07,13.7341,60322.2,145192,0,98696.7,180,1,...,1,0,0,0,0,0,0,1,0,0
8,11220009,12001008,9.49928e+06,18.9156,188502,178778,0,131790,180,0,...,1,0,0,0,1,0,0,0,0,0
9,11220010,12001009,7.75594e+06,15.6069,27732.8,120218,1,81720,180,1,...,1,0,0,0,0,0,0,0,0,1


In [182]:
lmsdg4_cleaned_predict = pd.merge(lms_dg4,test)
lmsdg4_cleaned_predict

,AGREEMENTID,CUSTOMERID,LOAN_AMT,ROI,PRE_EMI,EMI,EMI_PAID,TOTAL_PAID,TENURE,HL,...,SINGLE,DOCTRATE,ENG,GRAD,MAT,OTHERS,POSTGRAD,PRF,UG,FORECLOSURE
0,11220003,12001002,4.0535e+06,17.1676,0,68239.8,1,46387,180,1,...,0,0,0,1,0,0,0,0,0,NaN
1,11220004,12001003,1.3257e+07,15.6069,28441.7,194918,1,193884,180,1,...,0,0,0,1,0,0,0,0,0,NaN
2,11220005,12001004,1.32114e+07,16.5433,0,202948,1,137957,180,1,...,0,0,0,1,0,0,0,0,0,NaN
3,11220007,12001006,6.15531e+06,16.5433,451448,104347,1,70931.7,180,1,...,0,0,0,0,0,0,0,0,1,NaN
4,11220009,12001008,9.49928e+06,18.9156,188502,178778,0,131790,180,0,...,0,0,0,1,0,0,0,0,0,NaN
5,11220013,12001012,5.40066e+06,13.7341,5627.39,75269.7,0,51165.7,240,1,...,0,0,0,1,0,0,0,0,0,NaN
6,11220015,12001014,4.30872e+06,17.4797,24033.5,73971,0,50282.9,180,0,...,0,0,0,0,0,0,1,0,0,NaN
7,11220018,12001017,7.68864e+06,16.5433,0,118111,1,80287.8,180,1,...,0,0,0,0,0,0,0,0,1,NaN
8,11220019,12001018,1.7252e+07,13.7341,167786,269322,0,183076,156,1,...,0,0,0,1,0,0,0,0,0,NaN
9,11220023,12001022,1.13752e+07,19.0404,20453.4,230513,1,156695,120,0,...,0,0,0,1,0,0,0,0,0,NaN


In [183]:
lmsdg4_cleaned_predict.drop(['CUSTOMERID','AGREEMENTID','FORECLOSURE'], inplace = True, axis = 1)
lmsdg4_cleaned_predict

,LOAN_AMT,ROI,PRE_EMI,EMI,EMI_PAID,TOTAL_PAID,TENURE,HL,LAP,STHL,...,MARRIED,SINGLE,DOCTRATE,ENG,GRAD,MAT,OTHERS,POSTGRAD,PRF,UG
0,4.0535e+06,17.1676,0,68239.8,1,46387,180,1,0,0,...,1,0,0,0,1,0,0,0,0,0
1,1.3257e+07,15.6069,28441.7,194918,1,193884,180,1,0,0,...,1,0,0,0,1,0,0,0,0,0
2,1.32114e+07,16.5433,0,202948,1,137957,180,1,0,0,...,1,0,0,0,1,0,0,0,0,0
3,6.15531e+06,16.5433,451448,104347,1,70931.7,180,1,0,0,...,1,0,0,0,0,0,0,0,0,1
4,9.49928e+06,18.9156,188502,178778,0,131790,180,0,1,0,...,1,0,0,0,1,0,0,0,0,0
5,5.40066e+06,13.7341,5627.39,75269.7,0,51165.7,240,1,0,0,...,1,0,0,0,1,0,0,0,0,0
6,4.30872e+06,17.4797,24033.5,73971,0,50282.9,180,0,1,0,...,1,0,0,0,0,0,0,1,0,0
7,7.68864e+06,16.5433,0,118111,1,80287.8,180,1,0,0,...,1,0,0,0,0,0,0,0,0,1
8,1.7252e+07,13.7341,167786,269322,0,183076,156,1,0,0,...,1,0,0,0,1,0,0,0,0,0
9,1.13752e+07,19.0404,20453.4,230513,1,156695,120,0,1,0,...,1,0,0,0,1,0,0,0,0,0


In [187]:
lmsdg4_cleaned_predict.drop(['BRANCH_PINCODE'],inplace = True,axis = 1)

In [254]:
predictions_lmsdg = model.predict(lmsdg4_cleaned_predict)

In [255]:
predictions_lmsdg

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)